Lambda School Data Science

*Unit 1, Sprint 1, Module 3*

---

# Join and Reshape datasets

Objectives
- concatenate data with pandas
- merge data with pandas
-  understand tidy data formatting
-  melt and pivot data with pandas

Links
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf)
- [Tidy Data](https://en.wikipedia.org/wiki/Tidy_data)
  - Combine Data Sets: Standard Joins
  - Tidy Data
  - Reshaping Data
- Python Data Science Handbook
  - [Chapter 3.6](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html), Combining Datasets: Concat and Append
  - [Chapter 3.7](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html), Combining Datasets: Merge and Join
  - [Chapter 3.8](https://jakevdp.github.io/PythonDataScienceHandbook/03.08-aggregation-and-grouping.html), Aggregation and Grouping
  - [Chapter 3.9](https://jakevdp.github.io/PythonDataScienceHandbook/03.09-pivot-tables.html), Pivot Tables
  
Reference
- Pandas Documentation: [Reshaping and Pivot Tables](https://pandas.pydata.org/pandas-docs/stable/reshaping.html)
- Modern Pandas, Part 5: [Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [83]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-10-02 21:51:27--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.131.149
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.131.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’

instacart_online_gr 100%[===================>] 196.03M  73.6MB/s    in 2.7s    

2019-10-02 21:51:30 (73.6 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz’ saved [205548478/205548478]



In [84]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [85]:
%cd instacart_2017_05_01

/content/instacart_2017_05_01/instacart_2017_05_01/instacart_2017_05_01


In [86]:
!ls -lh *.csv

-rw-r--r-- 1 502 staff 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 502 staff  270 May  2  2017 departments.csv
-rw-r--r-- 1 502 staff 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 502 staff  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 502 staff 104M May  2  2017 orders.csv
-rw-r--r-- 1 502 staff 2.1M May  2  2017 products.csv


# Assignment

## Join Data Practice

These are the top 10 most frequently ordered products. How many times was each ordered? 

1. Banana
2. Bag of Organic Bananas
3. Organic Strawberries
4. Organic Baby Spinach 
5. Organic Hass Avocado
6. Organic Avocado
7. Large Lemon 
8. Strawberries
9. Limes 
10. Organic Whole Milk

First, write down which columns you need and which dataframes have them.

Next, merge these into a single dataframe.

Then, use pandas functions from the previous lesson to get the counts of the top 10 most frequently ordered products.

In [87]:
# 0th
# Import dataframes

import pandas as pd

# So that we have a visual reference for later
def viewdf(name, shape, head):
  print(name)
  display(shape)
  display(head)
  print('\n')

# DF FACTORY
def inidf(fn):
  df = pd.read_csv('%s.csv' % fn)
  viewdf(fn, df.shape, df.head())
  return df

# Basic
aisles = inidf('aisles')
departments = inidf('departments')
orders = inidf('orders')
products = inidf('products')
order_products__prior = inidf('order_products__prior')
order_products__train = inidf('order_products__train')

# Combine prior & train
order_products = pd.concat([order_products__prior, order_products__train])
viewdf('order_products', order_products.shape, order_products.head())

aisles


(134, 2)

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation




departments


(21, 2)

...

(1384617, 4)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1




order_products


(33819106, 4)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [0]:
# 1st
# List of columns needed,
# Dataframes that have them

# NEEDED: product_name
#         product_id
# DATAFRAME: products

# NEEDED: product_id
# DATAFRAME: order_products (ie, we'll count each occurance)

In [88]:
# 2nd
# Merge the dataframes from above

# I'm not sure if it's necessary, but I'm going to
# trim the dataframes to just the products
# that we're interested in

desired_names = ['Banana', 'Bag of Organic Bananas', 'Organic Strawberries',
                 'Organic Baby Spinach', 'Organic Hass Avocado',
                 'Organic Avocado', 'Large Lemon', 'Strawberries',
                 'Limes', 'Organic Whole Milk']

products['is_desired'] = products['product_name'].apply(
    lambda x: x in desired_names)
desired_products = products[products['is_desired']]

desired_ids = desired_products['product_id'].to_list()

order_products['is_desired'] = order_products['product_id'].apply(
  lambda x: x in desired_ids)
desired_orders = order_products[order_products['is_desired']]

# NOW we'll merge them:

orders_w_names = pd.merge(desired_orders, desired_products, on='product_id')
orders_w_names.head()

,order_id,product_id,add_to_cart_order,reordered,is_desired_x,product_name,aisle_id,department_id,is_desired_y
0,3,21903,4,1,True,Organic Baby Spinach,123,4,True
1,26,21903,6,0,True,Organic Baby Spinach,123,4,True
2,31,21903,3,1,True,Organic Baby Spinach,123,4,True
3,39,21903,4,0,True,Organic Baby Spinach,123,4,True
4,56,21903,8,1,True,Organic Baby Spinach,123,4,True


In [89]:
# TA DAAA

orders_w_names['product_name'].value_counts()

Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

## Reshape Data Section

- Replicate the lesson code
- Complete the code cells we skipped near the beginning of the notebook
- Table 2 --> Tidy
- Tidy --> Table 2
- Load seaborn's `flights` dataset by running the cell below. Then create a pivot table showing the number of passengers by month and year. Use year for the index and month for the columns. You've done it right if you get 112 passengers for January 1949 and 432 passengers for December 1960.

In [90]:
# 1st
# Recreate the lesson code

%matplotlib inline
import numpy as np
import seaborn as sns

table1 = pd.DataFrame(
    [[np.nan, 2],
     [16,    11], 
     [3,      1]],
    index=['John Smith', 'Jane Doe', 'Mary Johnson'], 
    columns=['treatmenta', 'treatmentb'])

table2 = table1.T

table2

,John Smith,Jane Doe,Mary Johnson
treatmenta,NaN,16.0,3.0
treatmentb,2.0,11.0,1.0


In [0]:
# 2nd
# Table 2 -> Tidy

tidy = table2.reset_index().melt(id_vars=['index'],
                                 value_vars=['John Smith',
                                             'Jane Doe',
                                             'Mary Johnson'],
                                 var_name='name',
                                 value_name='value')
tidy

,index,name,value
0,treatmenta,John Smith,NaN
1,treatmentb,John Smith,2.0
2,treatmenta,Jane Doe,16.0
3,treatmentb,Jane Doe,11.0
4,treatmenta,Mary Johnson,3.0
5,treatmentb,Mary Johnson,1.0


In [91]:
# 3rd
# Tidy -> Table 2

new_table2 = tidy.pivot(index='index', columns='name', values='value')
new_table2

name,Jane Doe,John Smith,Mary Johnson
index,,,
treatmenta,16.0,NaN,3.0
treatmentb,11.0,2.0,1.0


In [0]:
flights = sns.load_dataset('flights')

In [93]:
# 4th
# Pivot table of passengers per month and year

flights_pt = flights.pivot_table(index='year', columns='month')
display(flights_pt)

# Check our work

assert(flights_pt.loc[1949]['passengers']['January'] == 112)
assert(flights_pt.loc[1960]['passengers']['December'] == 432)

passengers                       ...                                    
month    January February March April  ... September October November December
year                                   ...                                    
1949         112      118   132   129  ...       136     119      104      118
1950         115      126   141   135  ...       158     133      114      140
1951         145      150   178   163  ...       184     162      146      166
1952         171      180   193   181  ...       209     191      172      194
1953         196      196   236   235  ...       237     211      180      201
1954         204      188   235   227  ...       259     229      203      229
1955         242      233   267   269  ...       312     274      237      278
1956         284      277   317   313  ...       355     306      271      306
1957         315      301   356   348  ...       404     347      305      336
1958         340      318   362   348  ...       404     359      310      337
1959         360      342   406   396  ...       463     407      362      405
1960         417      391   419   461  ...       508     461      390      432

[12 rows x 12 columns]

## Join Data Stretch Challenge

The [Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) has a visualization of "**Popular products** purchased earliest in the day (green) and latest in the day (red)." 

The post says,

> "We can also see the time of day that users purchase specific products.

> Healthier snacks and staples tend to be purchased earlier in the day, whereas ice cream (especially Half Baked and The Tonight Dough) are far more popular when customers are ordering in the evening.

> **In fact, of the top 25 latest ordered products, the first 24 are ice cream! The last one, of course, is a frozen pizza.**"

Your challenge is to reproduce the list of the top 25 latest ordered popular products.

We'll define "popular products" as products with more than 2,900 orders.



In [95]:
# filter out unpopular products

all_counts = pd.DataFrame.from_dict(
    order_products['product_id'].value_counts().to_dict(), orient='index')
most_popular = all_counts[all_counts[0] > 2900].reset_index()
most_popular.columns = ['product_id', 'count']
most_popular = order_products[order_products['product_id'].isin(
    most_popular['product_id'])]
viewdf('>2900 orders', most_popular.shape, most_popular.head())

# still 22M rows, oof

>2900 orders


(22450886, 5)

,order_id,product_id,add_to_cart_order,reordered,is_desired
0,2,33120,1,1,False
1,2,28985,2,1,False
2,2,9327,3,0,False
5,2,17794,6,1,False
9,3,33754,1,1,False


In [96]:
# Merge w/ orders,
# which has the hour_of_day field

popular_times = pd.merge(most_popular, orders, on='order_id')
viewdf('popular w/ times', popular_times.shape, popular_times.head())

popular w/ times


(22450886, 11)

,order_id,product_id,add_to_cart_order,reordered,is_desired,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,False,202279,prior,3,5,9,8.0
1,2,28985,2,1,False,202279,prior,3,5,9,8.0
2,2,9327,3,0,False,202279,prior,3,5,9,8.0
3,2,17794,6,1,False,202279,prior,3,5,9,8.0
4,3,33754,1,1,False,205970,prior,16,5,17,12.0


In [105]:
# How did they decide that one product was "later" than another?
# All I can think of is finding, for every product, the mean of
# order_hour_of_day, and then getting the 25 "latest" means

results = {}
for n in popular_times['product_id'].unique():
  df = popular_times[popular_times['product_id'] == n]
  results[n] = df['order_hour_of_day'].mean()
results = pd.DataFrame.from_dict(results, orient='index')
results.head()

# I'm just improvising here, super slow & in need of a refactor

,0
33120,13.428289
28985,13.577373
9327,13.591566
17794,13.479537
33754,13.104160


In [113]:
# Now just need the top 25 means...

latest_results = results.sort_values(0, ascending=False).head(25)
latest_results = latest_results.reset_index()['index'].to_list()

# As names, not ids...
actual_names = map(lambda x: products.loc[products['product_id'] == x]['product_name'].to_list()[0], latest_results)
display(list(actual_names))

# Holy cow, did I actually get the full list?!

['Half Baked Frozen Yogurt',
 'The Tonight Dough™ Ice Cream',
 'Americone Dream® Ice Cream',
 'Half Baked® Ice Cream',
 'Phish Food® Ice Cream',
 'Milk and Cookies Ice Cream',
 'Sicilian Pistachio',
 'Chocolate Peanut Butter Cup Gelato',
 'Birthday Cake Light Ice Cream',
 'Cherry Garcia Ice Cream',
 'Strawberry Ice Cream',
 'Vanilla Bean Light Ice Cream',
 'Coffee Ice Cream',
 'Roman Raspberry Sorbetto',
 'Mediterranean Mint Gelato',
 'Caramel Cookie Crunch Gelato',
 'Chocolate Chip Cookie Dough Ice Cream',
 'Mint Chip',
 'Coconut Almond Minis Frozen Dessert Bars',
 'Mint Chocolate Chip Ice Cream',
 'Chocolate Peanut Butter Ice Cream',
 'Vanilla Milk Chocolate Almond Ice Cream Bars',
 'Sea Salt Caramel Gelato',
 'Vanilla Bean Ice Cream',
 'Rising Crust Pepperoni Pizza']

## Reshape Data Stretch Challenge

_Try whatever sounds most interesting to you!_

- Replicate more of Instacart's visualization showing "Hour of Day Ordered" vs "Percent of Orders by Product"
- Replicate parts of the other visualization from [Instacart's blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2), showing "Number of Purchases" vs "Percent Reorder Purchases"
- Get the most recent order for each user in Instacart's dataset. This is a useful baseline when [predicting a user's next order](https://www.kaggle.com/c/instacart-market-basket-analysis)
- Replicate parts of the blog post linked at the top of this notebook: [Modern Pandas, Part 5: Tidy Data](https://tomaugspurger.github.io/modern-5-tidy.html)

In [116]:
# Get the 25 earliest as well

earliest_results = results.sort_values(0, ascending=False).tail(25)
earliest_results = earliest_results.reset_index()['index'].to_list()

# As names, not ids...
early_names = map(lambda x: products.loc[products['product_id'] == x]['product_name'].to_list()[0], earliest_results)
display(list(early_names))

['Cinnamon Brown Sugar Breakfast Biscuits',
 'Orange & Lemon Flavor Variety Pack Sparkling Fruit Beverage',
 'French Vanilla Coffee Creamer',
 'Half And Half Cream',
 'Clementines',
 'Original Almondmilk',
 'with Crispy Almonds Cereal',
 'Pink Lady Apples',
 'Extra Fancy Unsalted Mixed Nuts',
 '0% Greek Strained Yogurt',
 'Milk Chocolate Almonds',
 'Popcorn',
 'Cheez-It Cheddar Cracker',
 'Original Beef Jerky',
 "Crunchy Oats 'n Honey Granola Bars",
 'Zero Calorie Cola',
 'Dry Roasted Almonds',
 'Cheez-It Baked Snack Crackers',
 'Apples',
 'Original Rice Krispies Treats',
 'Trail Mix',
 'Nutri Grain Bars Multi Pack',
 'Pub Mix',
 'Sweet & Salty Nut Granola Bars Peanut',
 'Half And Half Ultra Pasteurized']

RangeIndex(start=0, stop=1, step=1)